In [1]:
from __future__ import print_function
import astropy
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#sys.path.pop(0)
%matplotlib inline

# First, lets make sure that barycorrpy is working

In [2]:
import barycorrpy
barycorrpy.__version__ #This tutorial assumes v0.4.4

'0.4.4'

In [3]:
# Running sample script - this should print:
# """
# ***********SUCCESS**************
# All barycentric correction velocities,  and time stamp conversions match expected values.
# """
from barycorrpy import sample_script 
results = sample_script.run_sample() 

Please join our Google Group to keep abreast of the latest versions and bug fixes - https://groups.google.com/g/barycorrpy
***********SUCCESS**************
All barycentric correction velocities,  and time stamp conversions match expected values.


# Import our package

In [4]:
import neidspec
neidspec

<module 'neidspec' from '/home/tehan/PycharmProjects/neidspec/neidspec/__init__.py'>

In [6]:
from astropy.io import fits
hdul = fits.open("/home/tehan/Documents/NeidSpecMatch/library/FITS/neidL2_20210805T053326.fits")
hdul[7].data[:, 0]

array([           nan,            nan,     0.        ,  3570.94367126,
        3591.94333894,  3613.47983754,  3635.09951646,  3656.83234655,
        3679.1573399 ,  3701.75997337,  3724.37146318,  3747.35868462,
        3770.57686586,  3794.19326882,  3818.06642302,  3842.23298328,
        3866.7091192 ,  3891.48660622,  3916.59850095,  3942.01710504,
        3967.80097092,  3993.90552408,  4020.36610681,  4047.16164987,
        4074.34994209,  4101.87150523,  4129.76985047,  4158.06178248,
        4186.73911121,  4215.83231735,  4245.29682955,  4275.21192486,
        4305.5225418 ,  4336.29124525,  4367.49423521,  4399.13940817,
        4431.26137395,  4463.8450136 ,  4496.91978452,  4530.4812161 ,
        4564.55096207,  4599.12550944,  4634.24637045,  4669.89780368,
        4706.10629927,  4742.87683511,  4780.22449168,  4818.16955921,
        4856.72206938,  4895.88966379,  4935.69832859,  4976.16662435,
        5017.30088343,  5059.10987254,  5101.63438257,  5144.89446693,
      

In [10]:
# Lets read in an HPF spectrum
# The function reads in the object name from the header
# If ccf_redshift is true, then it will redshift the wavelength to the stellar frame
# i.e., this takes out the absolute redshift estimated by calculating a CCF
N = neidspec.NEIDSpectrum("/home/tehan/Documents/NeidSpecMatch/library/FITS/neidL2_20210805T053326.fits",ccf_redshift=True)

/home/tehan/PycharmProjects/neidspec/neidspec/neidspec.py:288: RuntimeWarning: divide by zero encountered in divide
  self.f_sky_debl = self.hdu[2].data[s:e] * self.exptime / hdu[2].data[s:e]
/home/tehan/PycharmProjects/neidspec/neidspec/neidspec.py:288: RuntimeWarning: invalid value encountered in divide
  self.f_sky_debl = self.hdu[2].data[s:e] * self.exptime / hdu[2].data[s:e]
/home/tehan/PycharmProjects/neidspec/neidspec/neidspec.py:293: RuntimeWarning: invalid value encountered in divide
  self.f_debl[i] = self.f_debl[i] / np.nanmedian(self.f_debl[i])


In [11]:
# Barycentric julian date of the observation
N.bjd

2459431.7438520566

In [12]:
print(N) # The SNR is for HPF order 18 (~1micron)

NEIDSpec(TIC 188589164,sn55=4.8)


### This creates a 'Target' class instance within our object

- The config file for the object is by default saved to "../data/target_files/"
- If that file already exist, then it is read in from disk. If it does not exist, it queries Simbad and creates the file
- This way you can change the values within the file if it is already created

In [ ]:
# This is used 
N.target

In [ ]:
N.target.ra, N.target.dec

In [ ]:
# We can use this to calculate the barycentric velocity in km/s
t = astropy.time.Time("2019-10-08 00:00:00.0",format="iso").jd
N.target.calc_barycentric_velocity(t,obs='McDonald Observatory')

In [ ]:
# The barycentric velocity is automatically stored in an attribute
N.berv # km/s

In [ ]:
np.shape(N.f_debl)

## Lets plot the spectrum 

In [ ]:
N.plot_order(102,deblazed=False)

In [ ]:
N.plot_order(102,deblazed=True)

In [ ]:
N.plot_order(101,deblazed=True)

In [ ]:
#We could also this the following way using the handy wavelength and flux attributes
o = 102
o -= 10

# Blazed
fig, ax = plt.subplots()
ax.plot(N.w_shifted[o],N.f[o])
ax.set_title("Blazed spectrum shifted to stellar restframe\n(corrected for barycentric and absolute RV motion)")

# Deblazed
fig, ax = plt.subplots()
ax.plot(N.w_shifted[o],N.f_debl[o])
ax.set_title("Deblazed spectrum shifted to stellar restframe\n(corrected for barycentric and absolute RV motion)")


In [ ]:
# Other attributes are:
print(N.w_shifted.shape) # shifted wavelength
print(N.w.shape)         # non-shifted original wavelength
print(N.f.shape)         # non-deblazed science flux (sky-subtracted)
print(N.f_debl.shape)    # deblazed science flux (sky-subtracted)
print(N.f_sci.shape)     # Science flux (not sky-subtracted)
print(N.e.shape)         # error for deblazed sky-subtracted science flux
print(N.object)          # Object
print(N.bjd)             # BJD_TDB midpoint
print(N.sn55)            # SNR for order 55
print(N.berv)            # barycentric correction in km/s

# CCFs

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125.,125.,1501)
orders = [55,56,102]
ccf = N.calculate_ccf_for_orders(v,orders=orders,plot=True)

In [ ]:
fig, ax = plt.subplots()
o = 55
ax.plot(v,ccf[o - 10]/np.nanmedian(ccf[o - 10]))

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125., 125., 1501)
orders = [55, 56, 102]
ccf = N.calculate_ccf_for_orders(v, orders=orders, plot=True)# Calculates on barycentric shifted (not abs RV shifted) and undeblazed version
# rv1 is just the argmin of the CCF, the rv2 is an actual Gaussian fit minimum to the CCF valley
# You can see that order 18 is pretty wonky -- likely tellurics
# orders = [55,56]
# rv1, rv2 = N.rvabs_for_orders(v,orders,plot=True,verbose=True)

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125.,125.,1501)
orders = [55,56,57,94,95,]
ccf = N.calculate_ccf_for_orders(v,orders=orders,plot=True)

# Working with a collection of spectra

In [ ]:
import glob

In [ ]:
files = glob.glob("../neidspec/data/neid/spectra/*.fits")
HS = neidspec.NEIDSpecList(filelist=files)

In [ ]:
HS.df

In [ ]:
HS.sn55

In [ ]:
HS.objects

In [ ]:
N